# 0.5_ss_chap5

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%reload_ext autoreload

In [4]:

from src import utils
import itertools
import logging
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [5]:

log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=log_fmt)
# logging.basicConfig(level=logging.DEBUG, format=log_fmt)
logger = logging.getLogger()

## Backward propagation

In [14]:
class MulLayer:
        def __init__(self):
                ''' Multiplication interface for backward propagation
                Multiplication returns f(x,y) =x * y
                The chain rule stands: df/dx = y, df/dy = x 
                Backward propagation is: dx = dout * df/dx = dout * y
                Backward propagation is: dy = dout * df/dy = dout * x
                
                Properties
                x: num
                        Forward x
                y: num
                        Forward y
                '''
                self.x = None
                self.y = None
        
        def forward(self, x, y):
                ''' Forward method of multiplication
                Initalized x, y on forward
                '''
                self.x = x
                self.y = y
                out = x * y
                
                return out
        
        def backward(self, dout):
                ''' Backward method of multiplication
                Chain rule follows that df/dx=y, df/dx=x
                '''
                dx = dout * self.y
                dy = dout * self.x
                
                return dx, dy


In [15]:
apple = 100
apple_num = 2
tax = 1.1

In [16]:
# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [17]:
# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

In [18]:
print(price)

220.00000000000003


In [19]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)

In [20]:
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [21]:
print(dapple)

2.2


In [22]:
print(dapple_num)

110.00000000000001


## Add

In [23]:
class AddLayer:
        ''' add interface for backward propagation
        add returns f(x,y) =x + y
        The chain rule stands: df/dx = 1, df/dy = 1 
        Backward propagation is: dx = dout * df/dx = dout * 1
        Backward propagation is: dy = dout * df/dy = dout * 1

        Properties
        x: num
                Forward x
        y: num
                Forward y
        '''
        def __init__(self):
                pass
        
        def forward(self, x, y):
                out = x + y
                return out
        
        def backward(self, dout):
                dx = dout * 1
                dy = dout * 1
                return dx, dy

In [24]:
orange = 150
orange_num = 3

In [26]:
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()

In [27]:
orange_price = mul_orange_layer.forward(orange, orange_num)

In [28]:
all_price = add_apple_orange_layer.forward(apple_price, orange_price)

In [29]:
price = mul_tax_layer.forward(all_price, tax)

### backward

In [31]:
dprice = 1

In [32]:
dall_price, dtax = mul_tax_layer.backward(dprice)

In [34]:
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)

In [35]:
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

In [36]:
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [37]:
print(price)

715.0000000000001


In [38]:
dapple_num

110.00000000000001

In [40]:
dorange_num

165.0

## Activation

### Relu

In [43]:
class Relu:
        ''' Relu activation
        f(x) = x (x > 0), 0 (x <=0)
        df/dx = 1(x>0), 0(x<=0)
        '''
        def __init__(self):
                self.mask = None
        
        def forward(self, x):
                ''' output relu
                
                Parameters
                ----------
                x: numpy array
                
                Returns
                -------
                out: numpy array
                        masked as 0 where x<=0
                '''
                self.mask = (x <= 0)
                out = x.copy()
                out[self.mask] = 0
                
                return out
        
        def backward(self, dout):
                dout[self.mask] = 0
                dx = dout
                
                return dx

### Sigmoid

In [44]:
class Sigmoid:
        ''' Sigmoid interface
        y = f(x) = 1/(1+exp(-x))
        dy/dx = exp(-x)/(1+exp(-x))^2 = y(1-y)
        '''
        def __init__(self):
                self.out = None
        
        def forward(self, x):
                out = 1/(1+np.exp(-x))
                self.out = out
                
                return out
        
        def backward(self, dout):
                dx = dout * (1.0 - self.out) * self.out
                
                return dx

In [ ]:
class Affine:
        ''' Interface for affine transformation (dot product)
        f(x, W, b) = x %*% W + b
        
        '''
        def __init__(self, W, b):
                self.W = W
                self.x = None
                self.dW = None
                self.db = None
        
        def forward(self, x):
                self.x = x
                out = np.dot(x, self.W) + self.b
                
                return out
        
        def backward(self, dout):
                dx = np.dot(dout, self.W.T)
                self.dW = np.dot(self.x.T, dout)
                self.db = np.sum(dout, axis=0)
                
                return dx